In [1]:
import wbdata
import pandas as pd
import numpy as np
import datetime
import time



def population_dataframes(countries="all", year=[2010]):
	def dissection_country(x,default2 = 0):
		dc_country = []
		



		

		x = np.array(x.replace(" ","").lower().split(',')) if ',' in x else np.array([x.replace(" ","").lower()])
		temp_test = np.zeros(len(x))
		temp_test2 = np.zeros(len(x))

		for i in population_dataframes.country:
			
			if True in (np.char.find(np.array(i['name']), x) > -1) or True in  (np.char.find(np.array(i['id']), x)>-1):
				
				dc_country += [i]
				temp_test[np.char.find(np.array(i['name']), x) > -1] += 1
				temp_test2[np.char.find(np.array(i['id']), x) > -1] += 1   #edit needed

	
		if False not in (temp_test == 1) or False not in (temp_test2 == 1) or False not in np.logical_or(temp_test == 1 ,temp_test2 == 1) :
			if default2 != 0:
				return [i['name'] for i in dc_country] if len(x)>1 else dc_country[0]['name']


			return [i['id'].upper() for i in dc_country] if len(x)>1 else dc_country[0]['id'].upper()


				
		else:
			raise Exception('please be more specific / you have typos')

	try:
		countries = dissection_country(countries) 
		data_dates = datetime.datetime(year[0], 1, 1) if len(year) == 1 else (datetime.datetime(year[0], 1, 1), datetime.datetime(year[1], 1, 1))

		#return population_dataframes.testdic
		return wbdata.get_dataframe(population_dataframes.testdic, data_date = data_dates, country = countries)


	except AttributeError:
		population_dataframes.test = np.array([i for i in wbdata.api.search_indicators('population', source = 16,display = False) if 'Male population' in i['name'] or 'Female population' in i['name'] or i['name']== "population ages 80 and above, female" ])
		population_dataframes.country = wbdata.get_country(display = False)
		population_dataframes.country_name = np.array([])
		for i in population_dataframes.country:
			i['name'] = i['name'].replace(" ","").lower()
			i['id'] = i['id'].lower()
			if i['name'] in ['korea,rep.','korea,dem.people���srep.','congo,dem.rep.', 'congo,rep.' ]:
				i['name'] = [k for j,k in zip(['korea,rep.','korea,dem.people���srep.','congo,dem.rep.', 'congo,rep.' ], ['northkorea','southkorea', 'democraticrepublicofthecongo','republicofthecongo'   ]) if j== i['name']][0]


			if ',' in  i['name'] and 'excludingsub-saharanafrica' not in i['name']:

				i['name'] = i['name'][:[ k for (j,k) in zip(i['name'],range(len(i['name']))) if j ==","][0]]
			
		population_dataframes.country_id = np.array([i['id'] for i in population_dataframes.country])

		population_dataframes.testdic = { i['id']:i['name'] if i['name'] != "population ages 80 and above, female" else 'Female population 80+'for i in population_dataframes.test}
		
		population_dataframes.testdic2 = { i['name']:i['id'] for i in population_dataframes.country}
		
		population_dataframes.age_range2 = {i:{ population_dataframes.testdic[j].split()[-1].replace(" ","") :j  for j in population_dataframes.testdic if i in population_dataframes.testdic[j]  }       for i in ['Female','Male']     } 


		population_dataframes.age_range = ([population_dataframes.testdic[i][16:]for i in population_dataframes.testdic if 'Male' in population_dataframes.testdic[i]])


		temp = np.array([population_dataframes.testdic[i] for i in population_dataframes.testdic])


		temp.shape = (len(temp),1)


	 	
		return population_dataframes(countries = countries,year= year)


In [2]:
##Based on the dataframe construction, pop
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np

def pop_pyramid(data, country, year):
    """this is a function taking pandas dataframe, one country and one year
    output is population pyramid graph of the country in the year"""
    py.init_notebook_mode(connected=True)
    age_range = ['0004','0509','1014','1519',
                 '2024','2529','3034','3539',
                 '4044','4549','5054','5559',
                 '6064','6569','7074','7579',
                 '80UP']
    female_bins = np.array(data.filter(regex = 'Female').loc[(country,year)])
    male_bins = np.array(data.filter(regex = 'Male').loc[(country,year)])
    female_bins = female_bins[::-1]
    male_bins = -male_bins[::-1]
    
    # set the plot
    layout = go.Layout(barmode='overlay',
                       title = country +' '+ year +' Population',
                       yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                       xaxis=go.layout.XAxis(title='Number'))
    bins = [go.Bar(x = male_bins,
               y = [int(s[:2])+1 for s in age_range],
               orientation='h',
               name='Men',
               marker=dict(color='pink'),
               hoverinfo='skip'
               ),

        go.Bar(x = female_bins,
               y = [int(s[:2])+1 for s in age_range],
               orientation='h',
               name='Women',
               marker=dict(color='lightblue'),
               hoverinfo='skip',
               )
        ]

    return py.iplot(dict(data=bins, layout=layout))
    
    
    
    
    
    

In [3]:
test_data = population_dataframes(countries = 'USA,China',year= [2010,2014])
test_data

Male population 80+   Male population 75-79  \
country       date                                                
China         2014             9577122.0             11554005.0   
              2013             9275564.0             11337555.0   
              2012             8862950.0             11063343.0   
              2011             8346717.0             10756096.0   
              2010             7746539.0             10430332.0   
United States 2014             4472925.0              3544764.0   
              2013             4409804.0              3424121.0   
              2012             4336428.0              3313172.0   
              2011             4239113.0              3223881.0   
              2010             4111181.0              3168819.0   

                    Female population 75-79  Male population 70-74  \
country       date                                                   
China         2014               12507385.0             16437254.0   
              2013               12319589.0             16168625.0   
              2012               12130164.0             16092418.0   
              2011               11937644.0             16089415.0   
              2010               11727557.0             16053779.0   
United States 2014                4323680.0              4887147.0   
              2013                4218243.0              4705865.0   
              2012                4146202.0              4574815.0   
              2011                4106572.0              4462984.0   
              2010                4104388.0              4354473.0   

                    Female population 70-74   Male population 65-69  \
country       date                                                    
China         2014                16469707.0             23791161.0   
              2013                16122595.0             22458600.0   
              2012                15992367.0             21522778.0   
              2011                15954867.0             20832251.0   
              2010                15919729.0             20299264.0   
United States 2014                 5801355.0              7118451.0   
              2013                 5550186.0              6748544.0   
              2012                 5341610.0              6366256.0   
              2011                 5165902.0              6013536.0   
              2010                 5023365.0              5725678.0   

                    Female population 65-69  Male population 60-64  \
country       date                                                   
China         2014               23880960.0             38045173.0   
              2013               22340513.0             35824329.0   
              2012               21146136.0             33062570.0   
              2011               20205430.0             30399899.0   
              2010               19467169.0             28265707.0   
United States 2014                7985214.0              8941411.0   
              2013                7617678.0              8730671.0   
              2012                7236405.0              8522419.0   
              2011                6872446.0              8285116.0   
              2010                6554662.0              8009356.0   

                    Female population 60-64  Male population 55-59  \
country       date                                                   
China         2014               37406350.0             39616191.0   
              2013               35256275.0             40232309.0   
              2012               32562807.0             41273492.0   
              2011               29915711.0             41901874.0   
              2010               27709115.0             41586594.0   
United States 2014                9617628.0             10482012.0   
              2013                9413400.0             10287567.0   
              2012                9208547.0       

In [4]:
pop_pyramid(data=test_data, country='United States', year='2010')